In [14]:
import csv

compilation_file = 'spotify_taylor_compilation_info.csv'
album_file = 'spotify_taylor_album_info.csv'
single_file = 'spotify_taylor_single_info.csv'
output_file = 'spotify_taylor_all_info.csv'

# 读取 compilation 数据
compilation_data = []
with open(compilation_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    compilation_data = list(reader)

# 读取 album 数据
album_data = []
with open(album_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    album_data = list(reader)

# 读取 single 数据
single_data = []
with open(single_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    single_data = list(reader)

# 添加 "Album Type" 列并进行标记
compilation_data[0].append("Album Type")
for row in compilation_data[1:]:
    row.append("Compilation")

album_data[0].append("Album Type")
for row in album_data[1:]:
    row.append("Album")

single_data[0].append("Album Type")
for row in single_data[1:]:
    row.append("Single")

# 合并数据
merged_data = compilation_data + album_data[1:] + single_data[1:]

# 保存为新的 CSV 文件
with open(output_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(merged_data)

print(f"Data merged and saved as {output_file}")

Data merged and saved as spotify_taylor_all_info.csv


In [2]:
#比对100首歌的list里面是不是所有歌都在spotifydata里

import csv
from fuzzywuzzy import fuzz, process

# 读取track_data_final.csv文件
track_data_file = 'Top100_taylor_swift_songs_final.csv'
track_data_songs = []

with open(track_data_file, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # 跳过标题行
    for row in csv_reader:
        track_name = row[1]
        if track_name.strip():  # 检查字符串是否非空
            track_data_songs.append(track_name)

# 读取spotify_taylor_all_audio_data_with popularity.csv文件
spotify_data_file = 'spotify_taylor_all_info.csv'
spotify_data_songs = []

with open(spotify_data_file, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # 跳过标题行
    for row in csv_reader:
        track_name = row[1]
        if track_name.strip():  # 检查字符串是否非空
            spotify_data_songs.append(track_name)

# 找出没有匹配上的歌曲
unmatched_songs = []

for track_data_song in track_data_songs:
    if track_data_song.strip():  # 检查字符串是否非空
        match = process.extractOne(track_data_song, spotify_data_songs)
        similarity = match[1]
        if similarity < 80:  # 根据相似度阈值判断是否匹配
            unmatched_songs.append(track_data_song)

# 打印没有匹配上的歌曲
for song in unmatched_songs:
    print(song)

crazier


In [20]:
import spotipy
import csv
from spotipy.oauth2 import SpotifyClientCredentials

# 设置你的client id和client secret
client_id = '86f246517a7444c5ba2c1cae880cefb7'
client_secret = '9fe989b66beb47fb8326454772f7af74'

# 获取访问权限
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# 定义要合并的CSV文件名
filename = 'spotify_taylor_all_info.csv'

# 指定歌曲名称和艺术家名称
track_name = 'crazier'
artist_name = 'Taylor Swift'

# 使用搜索接口搜索歌曲
results = sp.search(q='track:' + track_name + ' artist:' + artist_name, type='track', limit=1)
if len(results['tracks']['items']) > 0:
    track = results['tracks']['items'][0]

# 提取歌曲信息
if results['tracks']['items']:
    track = results['tracks']['items'][0]
    track_id = track['id']
    track_name = track['name']
    track_popularity = track['popularity']
    album_name = track['album']['name']
    album_release_date = track['album']['release_date']
    album_type = track['album']['album_type'].capitalize()
    

    # 获取歌曲的音频特征
    audio_features = sp.audio_features(track_id)

    # 提取音频特征
    if audio_features:
        track_features = audio_features[0]
        tempo = track_features['tempo']
        energy = track_features['energy']
        key = track_features['key']
        duration_ms = track_features['duration_ms']
        loudness = track_features['loudness']
        speechiness = track_features['speechiness']
        acousticness = track_features['acousticness']
        danceability = track_features['danceability']
        instrumentalness = track_features['instrumentalness']
        liveness = track_features['liveness']
        mode = track_features['mode']
        time_signature = track_features['time_signature']
        valence = track_features['valence']
    else:
        tempo = ''
        energy = ''
        key = ''
        duration_ms = ''
        loudness = ''
        speechiness = ''
        acousticness = ''
        danceability = ''
        instrumentalness = ''
        liveness = ''
        mode = ''
        time_signature = ''
        valence = ''

    # 将数据存储到CSV文件
    with open(filename, 'a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)

        # 写入歌曲信息
        writer.writerow([track_id, track_name, track_popularity, album_name, album_release_date,
                         tempo, energy, key, duration_ms, loudness, speechiness,
                         acousticness, danceability, instrumentalness,
                         liveness, mode, time_signature, valence, album_type])

    print("歌曲信息已存储到CSV文件中。")
else:
    print("未找到匹配的歌曲。")

歌曲信息已存储到CSV文件中。


In [21]:
import csv
import re
from fuzzywuzzy import fuzz
from datetime import datetime

input_file = 'spotify_taylor_all_info.csv'
output_file = 'spotify_taylor_all_info.csv'

# 读取原始数据
data = []
with open(input_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    data = list(reader)

# 添加 "Original Date" 列并进行标记
data[0].append("Original Date")

def fuzzy_search_release_date(song_name, data):
    min_date = datetime.max.strftime('%Y/%m/%d')
    for row in data[1:]:
        track_name = row[1]
        pattern = r"(.+?)\s*(?:\(.+\))?$"
        match_song_name = re.match(pattern, song_name, re.IGNORECASE)
        match_track_name = re.match(pattern, track_name, re.IGNORECASE)
        if match_song_name and match_track_name:
            song_key = match_song_name.group(1)
            track_key = match_track_name.group(1)
            ratio = fuzz.ratio(song_key.lower(), track_key.lower())
            if ratio == 100:
                release_date = row[4]
                if release_date < min_date:
                    min_date = release_date
    if min_date == datetime.max.strftime('%Y/%m/%d'):
        min_date = ''
    return min_date

for row in data[1:]:
    song_name = row[1]
    original_date = fuzzy_search_release_date(song_name, data)
    row.append(original_date)

# 保存为新的 CSV 文件
with open(output_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Data processed and saved as {output_file}")

Data processed and saved as spotify_taylor_all_info.csv


In [23]:
import csv
from datetime import datetime

input_file = 'spotify_taylor_all_info.csv'
output_file = 'spotify_taylor_all_info.csv'

# 读取原始数据
data = []
with open(input_file, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    data = list(reader)

# 添加 "Phase" 列并进行标记
data[0].append("Phase")

for row in data[1:]:
    original_date = row[19] 
    if original_date:
        try:
            date_obj = datetime.strptime(original_date, '%Y-%m-%d')
            if date_obj.year >= 2014:
                phase = "Phase 2"
            else:
                phase = "Phase 1"
        except ValueError:
            phase = ""
    else:
        phase = ""

    row.append(phase)
    

# 保存为新的 CSV 文件
with open(output_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Data processed and saved as {output_file}")

Data processed and saved as spotify_taylor_all_info.csv
